# #Module3

explain analyze
select * from tpch1.orders o join tpch1.customer c on o.o_custkey=c.c_custkey;


select * from tpch1.orders o ;
select count(*) from tpch1.customer c ;
Gather Motion 4:1  (slice3; segments: 4)  (cost=0.00..15178.97 rows=11 879 777 width=266)
;
select count(*) from tpch1.customer
union all select count(*) from tpch1.customer_1 ;
explain analyze
select * from tpch1.orders o join tpch1.customer_1 c on o.o_custkey=c.c_custkey;

select gp_segment_id, count(*) from gp_dist_random('tpch1.customer') group by 1;
select gp_segment_id, count(*) from gp_dist_random('tpch1.customer_1') group by 1;

select  sum( distinct r_regionkey) from tpch1.region;

explain analyze
select * from tpch1.nation_ext n join tpch1.region_ext r on n.n_regionkey=r.r_regionkey;

explain analyze
select * from tpch1.orders_2 o join tpch1.customer_2 c on o.o_custkey=c.c_custkey;
----------------------------------------- model linear regression
--Создадим таблицу, в которой есть идентификаторы, значения того, что нужно предсказать и x1, x2.
CREATE TABLE regr_example (
   id int,
   y int,
   x1 int,
   x2 int
);
INSERT INTO regr_example VALUES
   (1,  5, 2, 3),
   (2, 10, 7, 2),
   (3,  6, 4, 1),
   (4,  8, 3, 4);
   
  --Потом применяем к ней функцию linregr_train из библиотеки MADlib:
  SELECT madlib.linregr_train (
   'regr_example',         -- source table
   'regr_example_model',   -- output model table
   'y',                    -- dependent variable
   'ARRAY[1, x1, x2]'      -- independent variables
);
--В результаты выводится краткий отчет с основными коэффициентами регрессии, необходимыми для анализа полученной модели:
SELECT * FROM regr_example_model;
--С помощью полученной модели можно строить предсказания. Для этого нужно использовать функцию linregr_predict
SELECT 
    regr_example.*,
    madlib.linregr_predict ( ARRAY[1, x1, x2], m.coef ) AS predict,
    y - madlib.linregr_predict ( ARRAY[1, x1, x2], m.coef ) AS residual
    FROM regr_example, regr_example_model m;
-------------------------------------------------------
select habitat_id, st_area(geom) from public.buowl_habitat
where habitat_id in (select habitat_id from public.buowl_habitat order by habitat_id limit 5)
order by st_area(geom);
select * from public.buowl_habitat order by habitat_id limit 5;

select * from public.linear_projects ;
with cte1 as (
select t1.gid, st_intersects(t1.geom, t2.geom) as s, count(*) over() from public.linear_projects t1
inner join public.linear_projects t2 on t1.gid=t2.gid
)
select * from cte1 where s = 'true' ;


# Module 3 Lesson 7 
-- Geometrical and space coordinates (PostGIS  extantion)
CREATE TABLE public.ais_geom_test (gid int4, geom geometry,
  name varchar(25) ) distributed RANDOMLY ;

INSERT INTO public.ais_geom_test ( gid, geom, name )
  VALUES (1, 'POLYGON((0 0 0,0 5 0,5 5 0,5 0 0,0 0 0))', '3D Square');
INSERT INTO public.ais_geom_test ( gid, geom, name )
  VALUES (2, 'LINESTRING((1 1 1,5 5 5,7 7 5))', '3D Line');
INSERT INTO public.ais_geom_test ( gid, geom, name )
  VALUES (3, 'MULTIPOINT((3 4,8 9))', '2D Aggregate Point');
INSERT INTO public.ais_geom_test ( gid, geom, name )
  VALUES (1, ST_Polygon(ST)GeomFromText('LINESTRING(75.15 29.53,77 30,77.6 29.5 75.15 29.53)'), 4326, '3D Poly');
  
 select * from public.ais_geom_test;
 SELECT geometrytype (geom), NAME FROM public.ais_geom_test;
 
SELECT ST_Polygon(
    ST_GeomFromText('LINESTRING(75.15 29.53,77 30,77.6 29.5, 75.15 29.53)'), 4326
); 
DROP TABLE geotest;
CREATE TABLE geotest (id INT4, name VARCHAR(32) );
SELECT AddGeometryColumn ('geotest','geopoint', 4326,'POINT',2);

INSERT INTO geotest (id, name, geopoint)
  VALUES (1, 'Olympia', ST_GeometryFromText('POINT(-122.90 46.97)', 4326)); 
INSERT INTO geotest (id, name, geopoint)
  VALUES (2, 'Renton', ST_GeometryFromText('POINT(-122.90 47.50)', 4326));
 INSERT INTO geotest (id, name, geopoint)
  VALUES (2, 'IstanbulHome', ST_GeometryFromText('POINT(41.037 28.9763)', 4326));
 UPDATE  geotest 
   SET geopoint = ST_GeometryFromText('POINT(28.9763 41.037)', 4326)
   WHERE name = 'IstanbulHome';
INSERT INTO geotest (id, name, geopoint)
  VALUES (2, 'IstanbulHome', ST_GeometryFromText('POINT(28.9763 41.037)', 4326));
 
SELECT name, ST_AsText(geopoint) FROM geotest;

SELECT geopoint FROM geotest WHERE name = 'IstanbulHome'; -- a Map

select true;

with t1 as (
select * from public.linear_projects
)
, t2 as (
select count(*) over(), count(distinct t1.gid) over(), t1.gid as gid1, t2.gid as gid2 from public.linear_projects t2
inner join t1 on st_intersects(t1.geom,t2.geom) = true 
and t1.gid != t2.gid)
select * from t2
;
and cast(t1.geom as VARCHAR(64)) != cast(t2.geom as VARCHAR(64))
; 

--Greenplum Text
SELECT 'a fat cat sat on a mat and ate a fat rat'::tsvector @@ 'cat & rats'::tsquery;
SELECT 'a fat cat sat on a mat and ate a fat rat'::tsvector @@ 'cat'::tsquery;
SELECT 
   to_tsvector('english', p_name) || to_tsvector('english', p_type) || to_tsvector('english', p_comment)
FROM tpch1.part 
LIMIT 50;
select * from tpch1.part;
SELECT 
   to_tsvector('english', p_name)
FROM tpch1.part 
LIMIT 50;
--The @@@ text search operator (deprecated since 6.0). This operator is currently a synonym for the @@ operator.
select * from public.articles;
select to_tsvector('english', body) @@ lower('Ryanair') ::tsquery from public.articles;
select to_tsvector('english', body) from public.articles;

--ts_rank(tsvector,tsquery) -- ranking text searches
select ts_rank(to_tsvector('english', body),to_tsvector('english', body) @@ lower('airline')) from public.articles;
SELECT heading, body, ts_rank_cd(to_tsvector('english', body), query) AS rank
FROM public.articles, to_tsquery('airline') query
--WHERE query @@ textsearch
ORDER BY rank desc;

--Highlighting Results ts_headline (text,tsquery) 
SELECT heading, ts_headline('english', body,
  to_tsquery('ryanair | wizz'))
 from public.articles;




In [24]:
from datetime import datetime, timedelta

# get current datetime
date = datetime.now()
print('Datetime is:', date, type(date))
date = datetime.now() + timedelta(days=-4)
print(date) 

# get day of week as an integer
x = date.weekday()
print('Day of a week is:', x)



Datetime is: 2023-01-20 23:24:56.375094 <class 'datetime.datetime'>
2023-01-16 23:24:56.376769
Day of a week is: 0


In [26]:
from datetime import datetime
def week_day():
    return datetime.now().weekday() # note, monday is 0, tuesday is 1... so sunday is 6
week_day()

4

In [4]:
# datetime.strptime(ds, "%Y-%m-%d").weekday()
from datetime import datetime
ds = datetime(2023, 1, 23)
def week_day():
    return ds.weekday() # note, monday is 0, tuesday is 1... so sunday is 6
week_day()

0